In [ ]:
import os
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm
import math

# Define functions to read TSV files
def read_tsv(filename, inf_ind, skip_1st=False, file_encoding="utf8"):
    extract_inf = []
    with open(filename, "r", encoding=file_encoding) as tsv_f:
        if skip_1st:
            tsv_f.readline()
        line = tsv_f.readline()
        while line:
            line_list = line.strip().split("\t")
            temp_inf = [line_list[ind] for ind in inf_ind]
            extract_inf.append(temp_inf)
            line = tsv_f.readline()
    return extract_inf

# Define a function that reads an amino acid feature file and generates a feature dictionary
def get_features(filename, f_num=15):
    f_list = read_tsv(filename, list(range(16)), True)
    f_dict = {}
    left_num = 0
    right_num = 0
    if f_num > 15:
        left_num = (f_num - 15) // 2
        right_num = f_num - 15 - left_num
    for f in f_list:
        f_dict[f[0]] = [0] * left_num + [float(x) for x in f[1:]] + [0] * right_num
    f_dict["X"] = [0] * f_num
    return f_dict

# Defining Input Functions
def generate_input(sps, sp_lbs, feature_dict, feature_num, ins_num, max_len):
    xs, ys = [], []
    i = 0
    for sp in sps:
        xs.append([[[0] * feature_num] * max_len] * ins_num)
        ys.append(sp_lbs[i])
        j = 0
        for tcr in sp:
            tcr_seq = tcr[0]
            right_num = max_len - len(tcr_seq)
            tcr_seq += "X" * right_num
            tcr_matrix = []
            for aa in tcr_seq:
                tcr_matrix.append(feature_dict[aa.upper()])
            xs[i][j] = tcr_matrix
            j += 1
        i += 1
    xs = np.array(xs)
    xs = xs.swapaxes(2, 3)
    ys = np.array(ys)
    return xs, ys

#Define the Generate Label function
def load_data(sample_dir):
    training_data = []
    training_labels = []
    for sample_file in os.listdir(sample_dir):
        training_data.append(read_tsv(os.path.join(sample_dir, sample_file), [0, 1], True))
        if "P" in sample_file:
            training_labels.append(1)
        elif "H" in sample_file:
            training_labels.append(0)
        else:
            print("Wrong sample filename! Please name positive samples with 'P' and negative samples with 'H'.")
            sys.exit(1)
    return training_data, training_labels

#Define the training function   
def train(fold, model, criterion, optimizer, train_loader, valid_loader, epoches=100, device="cuda"):
    model_path = f'../Model/{Cancer_name}_checkpoint_{fold}.pt'
    early_stopping = EarlyStopping(PATIENCE, path=model_path, verbose=False)

    epoch_train_losses = []
    epoch_test_losses = []
    with tqdm(total=epoches) as t:
        t.set_description(f'{Cancer_name} - Fold {fold}')
        for epoch in range(epoches):
            model.train()
            total_loss = 0.0
            for batch_x, batch_y in train_loader:
                
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)

                pred, attn = model(batch_x)
                loss = criterion(pred, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(train_loader)
            epoch_train_losses.append(avg_loss)
            test_avg_loss, _, _ = evaluate(model, criterion, valid_loader, device)
            epoch_test_losses.append(test_avg_loss)
            t.set_postfix(loss=avg_loss, test_loss=test_avg_loss)
            t.update(1)
            early_stopping(test_avg_loss, model)
            if early_stopping.early_stop:
                model.load_state_dict(torch.load(model_path))
                break

#Define the evaluation function
def evaluate(model, criterion, test_loader, device="cuda"):
    test_total_loss = 0.0
    all_preds = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for test_batch_x, test_batch_y in test_loader:
            test_batch_x = test_batch_x.to(device)
            test_batch_y = test_batch_y.to(device)
            test_pred, attn = model(test_batch_x)
            test_batch_y = test_batch_y
            loss = criterion(test_pred, test_batch_y)
            test_total_loss += loss.item()
            all_preds.append(test_pred.cpu().numpy())
            all_labels.append(test_batch_y.cpu().numpy())
        test_avg_loss = test_total_loss / len(test_loader)
    return test_avg_loss, all_preds, all_labels

#  a function to compute a binary classification indicator
def metrics(all_preds, all_labels, threshold=0.5):
    all_probs = np.array([1 / (1 + math.exp(-pred[1] + pred[0])) for pred in all_preds])
    binary_preds = (all_probs > threshold).astype(int)
    conf_matrix = confusion_matrix(all_labels, binary_preds)
    accuracy = accuracy_score(all_labels, binary_preds)
    sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    auc = roc_auc_score(all_labels, all_probs)

    return accuracy, sensitivity, specificity, auc

#Define BiFOmer 
class BiFormer(nn.Module):
    # BiFormer can predict a TCR repertoire's class (e.g. cancer-associated TCR repertoire) (binary classification).
    # It contains TCRD and the self-attention mechanism with topk for caTCRs identification.
    def __init__(self, tcr_num=100, aa_num=24, feature_num=15, kernel_size=None, filter_num=None, attention_head_num=1,
                 attention_hidden_size=10, drop_out=0.4, topk=0.05):
        super(BiFormer, self).__init__()
        self.tcr_num = tcr_num  # The number of TCR sequences that one input individual sample contains.
        self.aa_num = aa_num  # The number of amino acids that one TCR contains.
        self.feature_num = feature_num  # The dimension of the feature vector of one amino acid.
        if kernel_size is None:
            kernel_size = [2, 3, 4, 5, 6, 7]
        self.kernel_size = kernel_size  # The specification of the convolution kernel in the convolution layer.
        if filter_num is None:
            filter_num = [3, 3, 3, 2, 2, 1]
        assert len(filter_num) == len(kernel_size), \
            "The parameters 'kernel_size' and 'filter_num' set do not match!"
        self.filter_num = filter_num  # The number of the corresponding convolution kernels.
        self.all_filter_num = sum(self.filter_num)
        self.attention_head_num = attention_head_num
        self.attention_hidden_size = attention_hidden_size
        self.attention_head_size = int(self.attention_hidden_size / self.attention_head_num)
        self.drop_out = drop_out
        self.convs = nn.ModuleList([
            nn.Sequential(nn.Conv1d(in_channels=self.feature_num,
                                    out_channels=self.filter_num[idx],
                                    kernel_size=h,
                                    stride=1),
                          nn.ReLU(),
                          nn.AdaptiveMaxPool1d(1))
            for idx, h in enumerate(self.kernel_size)
        ])
        # Attention
        self.topk = int(self.tcr_num * topk)
        self.query = nn.Linear(self.all_filter_num, self.attention_hidden_size)
        self.key = nn.Linear(self.all_filter_num, self.attention_hidden_size)
        self.value = nn.Linear(self.all_filter_num, self.attention_hidden_size)
        self.attn_fc = nn.Linear(self.attention_hidden_size, self.all_filter_num)
        # Method I: Average #
        self.mil_fc = nn.Linear(self.all_filter_num, 2)
        # Method II: MLP #
        # self.fc = nn.Linear(self.attention_hidden_size, 1)
        # self.mil_fc = nn.Linear(self.tcr_num, 2)
        self.dropout = nn.Dropout(p=self.drop_out)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.attention_head_num, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, x):
        x = x.reshape(-1, self.feature_num, self.aa_num)
        out = [conv(x) for conv in self.convs]
        out = torch.cat(out, dim=1)
        out = out.reshape(-1, self.tcr_num, self.all_filter_num)
        # Attention
        mixed_query_layer = self.query(out)
        mixed_key_layer = self.key(out)
        mixed_value_layer = self.value(out)
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)
        # * Topk self-attention
        a_r = torch.matmul(query_layer.detach(), key_layer.detach().transpose(-1, -2))
        _, idx_r = torch.topk(torch.mean(a_r, 1), k=self.topk, dim=-1)
        # ** Compute topk self-attention
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
        idx_r = idx_r.unsqueeze(1).expand(-1, self.attention_head_num, -1, -1)
        value_layer = torch.gather(value_layer.unsqueeze(3).expand(-1, -1, -1, self.tcr_num, -1),
                                   dim=3, index=idx_r.unsqueeze(-1).expand(-1, -1, -1, -1, self.attention_head_size))
        attention_scores = torch.gather(attention_scores, dim=3, index=idx_r)
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        attention_probs = self.dropout(attention_probs)
        attention_probs = attention_probs.unsqueeze(-2)
        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.flatten(-2, -1)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.attention_hidden_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        context_layer = self.attn_fc(context_layer)
        # Pooling.
        # Method I #
        out = self.mil_fc(context_layer)
        # Avg pooling.
        out = torch.sum(out, dim=1) / self.tcr_num
        # Method II #
        # out = self.fc(hidden_states)
        # out = out.reshape(-1, self.tcr_num)
        # out = self.mil_fc(out)
        out = self.dropout(out)
        return out, attention_probs


# Setting model parameters   
def init_model():
    model = BiFormer(drop_out=0.4, attention_head_num=1, attention_hidden_size=10)
    return model


# Introduce an early stop mechanism
sys.path.append('../')
from python_codes.pytorchtools import EarlyStopping

# Read the amino acid characterization file
aa_file = "../Data/PCA15.txt"
aa_vectors = get_features(aa_file)

# 5-fold cross-validation
k_fold = 5
kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)

BATCH_SIZE = 64   # Batch size
NUM_EPOCHES = 2000 #Total number of training rounds
PATIENCE = 300    # Set the patience value for early stops
all_accuracies = []
all_sensitivities = []
all_specificities = []
all_aucs = []

device = "cuda"
# Cancers name
Cancer_list = ["BC","BL","UBC","BRCA","CC","CRC","DLBCL","GBM","HCC","Lung ADC","KS","Lung","MCC","Melanoma","MF","NSCLC","OS","OC","PDAC","PCa","RCC","SCLC"]
results = []
results_ROC = []

for Cancer_name in Cancer_list:
    data_dir = f'../Data/{Cancer_name}'
    training_data, training_labels = load_data(data_dir)
    print(f"Working on {Cancer_name} dataset: {len(training_data)} samples")

    all_preds = []
    all_labels = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(training_data)):
        train_data = [training_data[i] for i in train_idx]
        train_labels = [training_labels[i] for i in train_idx]
        test_data = [training_data[i] for i in test_idx]
        test_labels = [training_labels[i] for i in test_idx]

        train_data, valid_data, train_labels, valid_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=1234)

        train_input_batch, train_label_batch = generate_input(train_data, train_labels, aa_vectors, 15, 100, 24)
        train_input_batch, train_label_batch = torch.Tensor(train_input_batch).to(device), torch.LongTensor(train_label_batch).to(device)
        valid_input_batch, valid_label_batch = generate_input(valid_data, valid_labels, aa_vectors, 15, 100, 24)
        valid_input_batch, valid_label_batch = torch.Tensor(valid_input_batch).to(device), torch.LongTensor(valid_label_batch).to(device)
        test_input_batch, test_label_batch = generate_input(test_data, test_labels, aa_vectors, 15, 100, 24)
        test_input_batch, test_label_batch = torch.Tensor(test_input_batch).to(device), torch.LongTensor(test_label_batch).to(device)

        train_dataset = Data.TensorDataset(train_input_batch, train_label_batch)
        valid_dataset = Data.TensorDataset(valid_input_batch, valid_label_batch)
        test_dataset = Data.TensorDataset(test_input_batch, test_label_batch)

        train_loader = Data.DataLoader(train_dataset, BATCH_SIZE, True)
        valid_loader = Data.DataLoader(valid_dataset, BATCH_SIZE, True)
        test_loader = Data.DataLoader(test_dataset, BATCH_SIZE, True)

        model = init_model().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        train(fold, model, criterion, optimizer, train_loader, valid_loader, epoches=NUM_EPOCHES, device=device)

        
        _, preds, labels = evaluate(model, criterion, test_loader, device=device)
        all_preds += preds
        all_labels += labels

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    accuracy, sensitivity, specificity, auc = metrics(all_preds, all_labels)
    print(f"Mean Accuracy ({Cancer_name}): {accuracy:.4f}")
    print(f"Mean Sensitivity ({Cancer_name}): {sensitivity:.4f}")
    print(f"Mean Specificity ({Cancer_name}): {specificity:.4f}")
    print(f"Mean AUC ({Cancer_name}): {auc:.4f}")

    results.append({
        'cancer': Cancer_name,
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'auc': auc
    })

    results_ROC.append({
        'cancer': Cancer_name,
        'auc': auc,
        'all_preds': all_preds,
        'all_labels': all_labels
    })
